## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.00,71,0,16.55,PF,2022-02-13 06:21:18
1,1,Bambous Virieux,-20.3428,57.7575,86.25,74,40,14.97,MU,2022-02-13 06:21:19
2,2,Mount Gambier,-37.8333,140.7667,74.10,39,100,8.70,AU,2022-02-13 06:21:19
3,3,Nagato,34.3833,131.2000,46.35,87,100,4.27,JP,2022-02-13 06:21:20
4,4,Sistranda,63.7256,8.8340,41.90,73,100,16.49,NO,2022-02-13 06:12:26


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Min temperature you would like for your trip? "))
max_temp = float(input("Max temperature you would like for your trip? "))

Min temperature you would like for your trip? 80
Max temperature you would like for your trip? 110


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                           (city_data_df["Max Temp"] >= min_temp)]

city_data_df["Max Temp"]

0      79.00
1      86.25
2      74.10
3      46.35
4      41.90
       ...  
688    77.38
689    80.73
690    58.98
691    72.01
692    12.76
Name: Max Temp, Length: 693, dtype: float64

In [18]:
# 4a. Determine if there are any empty rows.
Temp_df.count()


City_ID       107
City          107
Lat           107
Lng           107
Max Temp      107
Humidity      107
Cloudiness    107
Wind Speed    107
Country       107
Date          107
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = Temp_df

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Temp_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Bambous Virieux,MU,86.25,-20.3428,57.7575,
8,Mahibadhoo,MV,81.00,3.7833,72.9667,
15,Labuhan,ID,84.38,-6.8844,112.2051,
25,Meulaboh,ID,85.32,4.1363,96.1285,
37,Kawalu,ID,80.82,-7.3817,108.2082,
39,Georgetown,MY,89.53,5.4112,100.3354,
40,Alofi,NU,80.49,-19.0595,-169.9187,
43,Busselton,AU,85.08,-33.6500,115.3333,
50,Finschhafen,PG,82.63,-6.6024,147.8681,
53,Kavieng,PG,82.27,-2.5744,150.7967,


In [41]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for row, column in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude=column["Lat"]
    Longitude=column["Lng"]
    
    params["location"] = f"{latitude},{Longitude}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[row, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))